# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [1]:
object StdDefinition {
    sealed abstract class IntList
    case class Empty() extends IntList
    case class NonEmpty(head:Int, tail: IntList) extends IntList
}
object StdDefinition2 {
    sealed abstract class List[A]
    case class Empty[A]() extends List[A]
    case class NonEmpty[A](head:A, tail: List[A]) extends List[A]
    
    type IntList = List[Int]
    

}
    val l: List[Int]=List(1,2,3)

defined object StdDefinition
defined object StdDefinition2
l: List[Int] = List(1, 2, 3)

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [1]:
object ActualStdDefinition{
    sealed abstract class List[+A]
    case class ::[A](head: A, tail: List[A]) extends List[A]
    case object Nil extends List[Nothing]
}
object ActualStdDefinition2{
    sealed abstract class List[+A]
    case class ::[A](head: A, tail: List[A]) extends List[A]
    case object Nil extends List[Nothing]
}

defined object ActualStdDefinition
defined object ActualStdDefinition2

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [2]:
// Less beautifully 
val l1: List[Int]=
    ::(1, ::(2, ::(3,Nil)))

// More idiomatically
val l2: List[Int]=
    1 :: 2 :: 3 :: Nil

val l3: List[Int]=
    List(1,2,3)

l1: List[Int] = List(1, 2, 3)
l2: List[Int] = List(1, 2, 3)
l3: List[Int] = List(1, 2, 3)

And can also pattern match lists, similarly:

In [3]:
// Less beautifully
l1 match {
    case Nil => 0
    case ::(head,tail)=>1
}

// more idiomatically
l1 match{
    case Nil =>0
    case head :: tail =>1
}

// or
l1 match{
    case List() => 0
    case List(h1,h2,h3) => 1
    case a => 2
}

res2_0: Int = 1
res2_1: Int = 1
res2_2: Int = 1

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [7]:
// Using mutable variables
def length[A](list: List[A]): Int  ={
    var out: Int = 0
    //for (e <- list)
     //   out = out + 1
    var aux:List[A] = list
    while(aux != Nil){
        out = out+1
        aux = aux.tail
    }
    out
}


defined function length

In [8]:
length(List(1,2,3,4,5))

res7: Int = 5

The recursive function is implemented as follows: 

In [13]:
// Using recursive functions
def lengthR[A](list: List[A]):Int =
    list match{
        case Nil => 0  
        case _ :: tail => 1 + lengthR(tail)
    }


defined function lengthR

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [3]:
// Using tail-recursive functions
def lengthTR[A](list: List[A]):Int ={
    
     @annotation.tailrec
    def lengthAux(out: Int, aux: List[A]): Int =
        aux match{
        case Nil => out  
        case _ :: tail => lengthAux(out+1,tail)
    }
    
    lengthAux(0,list)
}


defined function lengthTR

In [15]:
lengthTR(List(1,2,3,4,5))

res14: Int = 5

We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [ ]:
// First, imperatively

def constantList[A](value: A, length: Int): List[A] = {
    var acc: List[A] = Nil
    for (i <- 1 to length)
        acc = value :: acc
    acc
}

In [2]:
// Next, tail-recursively
def constantList[A](value: A, length: Int): List[A] = {

    def constantAux(acc: List[A], i: Int): List[A] = 
        if (i == 0) acc
        else constantAux(value :: acc, i-1)
    
    constantAux(Nil, length)
}


defined function constantList

Now, let's calculate the length of a list long enough, using each of the three implementations:

In [ ]:
// Imperatively
lengthI(constantList(0, 100000))

In [4]:
// Tail-recursive
lengthTR(constantList(0, 100000))

res3: Int = 100000

In [ ]:
// Plain recursive


### Unit testing with `scalatest`

In [11]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [17]:
class TestLength(lengthF: List[Int] => Int)
    extends FlatSpec with Matchers{ 
        "length" should "work" in{
            lengthF(List()) shouldBe 0
            lengthF(List(1)) shouldBe 1
            lengthF(List(1,2,3,4)) shouldBe 4
    }
}


defined class TestLength

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [19]:
run(new TestLength(lengthR))
run(new TestLength(lengthTR))

cmd16$Helper$TestLength:
length
- should work
cmd16$Helper$TestLength:
length
- should work


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [20]:
class TestSum(sum: List[Int] => Int)
    extends FlatSpec with Matchers{
        "sum" should "work" in {
            sum(Nil) shouldBe 0
            sum(List(5)) shouldBe 5
            sum(List(1,2,3,4)) shouldBe 10
        }
}

defined class TestSum

In [25]:
// Recursively
def sum(list: List[Int]): Int =
    list match{
        case Nil => 0 : Int
        case head :: tail => head +sum(tail) : Int
    }

defined function sum

In [27]:
run(new TestSum(sum))

cmd19$Helper$TestSum:
sum
- should work


In [30]:
// With tail-recursion

def sumR(list: List[Int]): Int ={
    
    def sumAux(aux: Int, list: List[Int]): Int =
        list match{
            case Nil => aux
            case head :: tail =>sumAux(aux+head,tail)
        }               
    sumAux(0,list) 
}

defined function sumR

In [28]:
run(new TestSum(sumR))

cmd19$Helper$TestSum:
sum
- should work


### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [ ]:
object TestProduct extends FlatSpec with Matchers{
    "product" should "work" in {
    }
}

This is the common recursive implementation:

It works as expected: 

In [ ]:
run(TestProduct)

But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
/*def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => head * product(tail)
    }
    */

In [ ]:
run(TestProduct)

A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [ ]:
object TestMember extends FlatSpec with Matchers{
    "member" should "work" in {
    }
}

In [ ]:
run(TestMember)

We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [10]:
class TestLast(last: List[Int] => Option[Int])
extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1)) shouldBe Some(1)
        last(List()) shouldBe None
    }
}

defined class TestLast

In [12]:
run( new TestLast(last))

cmd9$Helper$TestLast:
last
- should work


In [6]:
def last[A](list: List[A]) : Option[A] =
    list match {
        case Nil => 
            None : Option[A]
        case  head :: tail => //tail puede ser vacia o no vacia asi que tenemos que hacer pattern matching para saber que es 
            tail match {
                case Nil =>
                    Some(head) : Option[A]
                case head2 :: tail2 =>
                    last(tail) : Option[A]
            }
    }

defined function last

In [8]:
def last[A](list: List[A]) : Option[A] =
    list match {
        case Nil => 
            None : Option[A]
        case  head :: tail => 
            tail match {
                case Nil =>
                    Some(head) : Option[A]
                case _ :: _ =>
                    last(tail) : Option[A]
            }
    }

defined function last

In [9]:
def last[A](list: List[A]) : Option[A] =
    list match {
        case Nil => 
            None : Option[A]
        case  head :: tail => 
            if (tail.isEmpty) Some(head)
            else last(tail)
    }

defined function last

In [7]:
def last[A](list: List[A]) : Option[A] =
    list match {
        case Nil => 
            None : Option[A]
        case  head :: Nil => //tail puede ser vacia o no vacia asi que tenemos que hacer pattern matching para saber que es 
            Some(head) : Option[A]
        case head:: tail =>
            last(tail) : Option[A]
            
    }

defined function last

In [13]:
def last[A]: List[A] => Option[A] =
    {
        case Nil => 
            None 
        case  head :: Nil => //tail puede ser vacia o no vacia asi que tenemos que hacer pattern matching para saber que es 
            Some(head)
        case _ :: tail =>
            last(tail)
            
    }

defined function last

### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [15]:
class TestInsertLast(insertLast : (List[Int], Int) => List[Int]) 
extends FlatSpec with Matchers{
    "insertLast" should "work" in {
        insertLast(List(),1) shouldBe (List(1) : List[Int])
        insertLast(List(1),2) shouldBe (List(1,2): List[Int])
        insertLast(List(1,2,3,4),5) shouldBe (List(1,2,3,4,5) : List[Int])
    }
}

defined class TestInsertLast

In [17]:
def insertLast[A](list :List[A], a: A) : List[A]=
    list match{
        case Nil => List(a)
        case head :: tail =>  head :: insertLast(tail, a)
    }

defined function insertLast

In [18]:
run( new TestInsertLast(insertLast))

cmd14$Helper$TestInsertLast:
insertLast
- should work


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [19]:
class TestReverse(reverse: List[Int] => List[Int]) extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(List()) shouldBe List()
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
    }
}

defined class TestReverse

In [22]:
// Recursively: Really inefficient 
def reverseR[A](list: List[A]): List[A]=
    list match{
        case Nil =>
            Nil : List[A]
        case head :: tail =>
            reverseR(tail):+head : List[A]
    }

defined function reverseR

In [24]:
// Recursively: Really inefficient 
def reverseR[A](list: List[A]): List[A]=
   list match {
        case Nil =>
            Nil 
        case head :: tail =>
            reverseR(tail):+head 
    }

defined function reverseR

In [23]:
run(new TestReverse(reverseR))

cmd18$Helper$TestReverse:
reverse
- should work


In [25]:
def reverseR[A](list: List[A]): List[A]= {
    var out: List[A] = List()
    var aux: List[A] = list
    while(aux!=Nil){
        out =aux.head :: out
        aux=aux.tail
    }
    out    
}

defined function reverseR

In [27]:
// Tail-recursive, efficiently
def reverseTR[A](list: List[A]): List[A]= {
    def revAux(out:List[A], aux:List[A]): List[A] =
        aux match{
            case Nil =>out
            case head :: tail =>
                revAux(head :: out, tail)
        }
    revAux(List(),list)
}

defined function reverseTR

In [26]:
run(new TestReverse(reverseR))

cmd18$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)